In [ ]:
%pylab inline

import os
import numpy as np

from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import transforms

In [ ]:
train_dataset = FaceDataset()
val_dataset = FaceDataset()

train_data_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=1)

In [ ]:
from torchvision.models import resnet34

class SomeNet(torch.nn.Module):
    def __init__(self, num_classes=1, anchors):
        super().__init__()
        
        self.encoder = resnet34(pretrained=True)
        
        self.relu = nn.ReLU(inplace=True) 
        
        # TODO more channels
        self.conv1 = nn.Conv2d(512, 512, 3, padding=1, stride=2)
        self.bn1 = nn.BatchNorm2d(512)
        self.conv2 = nn.Conv2d(512, 512, 3, padding=1, stride=2)
        self.bn1 = nn.BatchNorm2d(512)
        self.conv3 = nn.Conv2d(512, 1024, 1)
        self.bn2 = nn.BatchNorm2d(1024)
        
        self.last = nn.Conv2d(1024, (5+num_classes)*len(anchors), 1)
        
        # TODO: multi-head later
        #self.conv_reg1 = nn.Conv2d(256, 5, 3, padding=1)
        #self.conv_class1 = nn.Conv2d(256, bottleneck, 1)
        #self.bn_class = nn.BatchNorm2d(bottleneck)
        #self.conv_class2 = nn.Conv2d(bottleneck, num_classes, 1)
    
    def forward(self, x):
        blocks = []
        
        #-> 3 640*640
        
        x = self.encoder.conv1(x)
        x = self.encoder.bn1(x)
        x = self.encoder.relu(x)
        x = self.encoder.maxpool(x)
        
        x = self.encoder.layer1(x) 
        x = self.encoder.layer2(x)        
        x = self.encoder.layer3(x)        
        x = self.encoder.layer4(x)
        #-> 512 20*20
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn2(x)
        x = self.relu(x)
        #-> 1024 5*5
        
        x = self.last(x)
        return x

In [ ]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')

net = SomeNet().to(device)